# Optimise Facial Recognition


## Giới thiệu

Đoạn code được trình bày trong Jupyter Notebook dưới đây gồm hai function:

- `embedding_extract`: Tạo ra các embedding từ các ảnh đã được cho trước
- `analyze`: Dựa vào các embedding được tạo ở function `embedding_extract`, analyze những khuôn mặt xuất hiện trong một video có sẵn

Hai function này tương ứng với hai method cùng tên của class Analyzer, và code được trình bày trong Jupyter Notebook này gần tương tự với code trong class Analyzer.

## Import

Đầu tiên, chúng ta import một số module cần thiết cho việc vận hành của notebook này.

In [8]:
%load_ext autoreload
%autoreload 2

In [10]:
import insightface
from insightface.app import FaceAnalysis
from imutils import paths
import cv2
import os
from annoy import AnnoyIndex
import time
import uuid
from tqdm import tqdm
from vidgear.gears import CamGear, WriteGear
import time


## Biến global

Sau đó, chúng ta thiết lập một số biến global cần thiết như sau.

In [18]:
f = 512
annoy = AnnoyIndex(f, 'angular')
face_model = insightface.app.FaceAnalysis()
face_model.prepare(ctx_id=-1, det_size=(640, 640))
face_model.models = {key: value for key, value in face_model.models.items() if key in ['detection', 'recognition']}
id_mapping = {}
name_mapping = ""
DETECT_THRESHOLD = 0.6

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

Ở đây, sau khi thiết lập `face_model`, ta dùng một dictionary comprehension và giữ lại models detection và recognition. Việc chỉ giữ lại hai model này sẽ giúp quá trình phân tích khuân mặt trong hàm `analyze` nhanh hơn rất nhiều, vì `face_model` giờ đây chỉ cần tập trung vào việc nhận diện và nhận dạng khuôn mặt đã cho trước, thay vì cần phải phân tích cảnh hay phân tích độ tuổi / giới tính.

Trong đoạn code được chứa trong `video.py`, những biến global này tương ứng với những attribute của class Analyzer.


## Hàm `extract_embedding`

Hàm này có hai tham số truyền vào như sau:

- `imgs_folder`: Thư mục chứa ảnh của những nhân vật chúng ta cần nhận dạng
- `id2name`: Giá trị mới của `name_mapping`


In [19]:
def embedding_extract(imgs_folder, id2name):
    name_mapping = id2name
    count = 0

    for img_path in tqdm(paths.list_images(imgs_folder)):
        try:
            dossier_id = img_path.split(os.path.sep)[-2]
            img = cv2.imread(img_path)
            img = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
            faces = face_model.get(img)
            if len(faces) == 1:
                for face in faces:
                    if face.det_score < DETECT_THRESHOLD:
                        continue
                    embedding = face.embedding.flatten()
                    annoy.add_item(count, embedding)
                    id_mapping[str(count)] = dossier_id
                    count += 1
        except Exception as e:
            continue

    annoy.build(200)
    print("[DEBUG] extract successful")

In [20]:
embedding_extract("input/hamilton", "hamilton")

2it [00:00,  2.74it/s]

[DEBUG] extract successful


In [21]:
print(id_mapping)

{'0': 'alex', '1': 'lin-manuel'}


In [22]:
print(annoy)

Hàm này sẽ thêm những cặp key-value vào `id_mapping`. Các cặp key-value sẽ có cấu trúc như sau:

- `key`: 0, 1, 2, ...
- `value`: Tên của những nhân vật được định dạng

Để gắn tên `X` với một ảnh `p`, ta đặt `p` vào thư mục con của `imgs_folder` có tên `X`. Trong ví dụ mà ta chạy trong notebook này, ảnh của nhân vật có tên `lin-manuel` được đặt trong thư mục `hamilton/lin-manuel`, và ảnh của nhân vật có tên `alex` được đặt trong thư mục `hamilton/alex`.

## Hàm analyze

Hàm này sẽ nhận vào 5 tham số như sau:

- `video_path`: đường dẫn tới video ta cần phân tích
- `sub_dir`: thư mục chứa những ảnh dossier được tìm thấy trong quá trình phân tích video
- `video_id`: id của video chúng ta cần phân tích, giá trị mặc định là `""`
- `outputs_video`: một boolean với giá trị mặc định là `False`. Nếu như giá trị của boolean này là `True`, ta sẽ xuất ra một video tên là output.mp4 (khi chạy từ notebook này) hoặc trong thư mục outputs (khi chạy từ method `analyze` của class `Analyzer`), với khuôn mặt của các nhân vật được bao trong một hình chữ nhật viền xanh lá cây. 
- `output_folder`: một string với giá trị mặc định là None. Sau khi video output được tạo xong, video output đó sẽ được lưu tại thư mục này. Trong trường hợp biến này có giá trị là None, tên của thư mục sẽ có giá trị mặc định là `output`.

Hàm này sẽ trả về một list các tuple. Mỗi tuple trong hàm này có 3 phần như sau:

- Tên của nhân vật được nhận dạng
- Đường dẫn tới ảnh dossier
- Thời gian nhân vật xuất hiện lần đầu trên màn hình

Bên cạnh đó, hàm này cũng sẽ xuất ra một file `dossier.log` ghi lại thời điểm mỗi nhân vật xuất hiện và rời khỏi màn hình.

In [27]:
def analyze(video_path, sub_dir, video_id='', outputs_video=False, output_folder=None):
    results = []

    status = 'PROCESSING'

    cap = CamGear(source=video_path).start()
    frame_rate = cap.framerate

    output_destination = output_folder if output_folder else 'output'

    if not os.path.exists(output_destination):
        os.mkdir(output_destination)

    if outputs_video:
        font = cv2.FONT_HERSHEY_DUPLEX
        video_file = video_path.split('/')[-1]
        video_name_tokens = video_file.split('.')[:-1]
        video_name = ".".join(video_name_tokens)
        print('video_name:', video_name)

        output_movie = WriteGear(
            output_filename=f"./{output_destination}/{video_name}_output.mp4")

    start_time = time.time()

    captured = 0
    frame_id = 0
    on_screen = set()

    with open('dossier.log', 'w') as f:
        try:
            while 1:
                frame = cap.read()

                if frame is None:
                    print('end')
                    break

                frame_id += 1

                if frame_id % 5 == 0:
                    captured += 1
                    small_frame = cv2.resize(
                        frame, (0, 0), fx=0.25, fy=0.25)
                    faces = face_model.get(small_frame)

                    if len(faces) <= 0:
                        if outputs_video:
                            output_movie.write(frame)
                        continue

                    if len(faces) > 0:
                        timestamp = time.time() - start_time
                        print(
                            f'[{frame_id}] identified {len(faces)} faces at {timestamp}')

                    seen_faces = set()

                    for face in faces:
                        if face.det_score < 0.6:
                            continue

                        # NOTE - bounding box for the face
                        left, top, right, bottom = tuple(
                            face.bbox.astype(int).flatten())

                        embedding = face.embedding.flatten()
                        annoy_start = time.time()
                        closest_indices, distances = annoy.get_nns_by_vector(
                            embedding, n=1, include_distances=True)
                        annoy_end = time.time()
                        print(
                            f"[{frame_id}] annoy finished in {annoy_end - annoy_start}")
                        similarity = (2. - distances[0] ** 2) / 2.

                        if similarity >= 0.2:
                            crop_img = small_frame[top:bottom, left:right]
                            dossier_id = id_mapping[str(
                                closest_indices[0])]
                            seen_faces.add(dossier_id)

                            if dossier_id not in on_screen:
                                print(
                                    f'{dossier_id} on screen since {frame_id / frame_rate}', file=f)
                                on_screen.add(dossier_id)
                                filename = dossier_id + '_' + \
                                    str(uuid.uuid4()) + '.jpg'
                                path_to_save = os.path.join(
                                    sub_dir, filename)
                                writer = WriteGear(
                                    output_filename=path_to_save, compression_mode=False)
                                writer.write(crop_img)
                                timecode = frame_id / frame_rate
                                results.append((dossier_id, path_to_save, timecode))

                            if outputs_video:
                                cv2.rectangle(frame, (left * 4, top * 4),
                                                (right * 4, bottom * 4), (0, 255, 0), 2)
                                cv2.rectangle(frame, (left * 4, bottom * 4 + 10),
                                                (right * 4, bottom * 4), (0, 255, 0), cv2.FILLED)
                                font = cv2.FONT_HERSHEY_DUPLEX
                                cv2.putText(frame, dossier_id, (left * 4 + 6, bottom * 4 - 6),
                                            font, 0.5, (255, 255, 255), 1)

                    for person in on_screen:
                        if person not in seen_faces:
                            print(
                                f'{person} left screen at {frame_id / frame_rate}', file=f)
                            # del tagged_trackers[person]

                    on_screen = seen_faces

                    if outputs_video:
                        output_movie.write(frame)
        finally:
            cap.stop()

            if outputs_video:
                output_movie.close()
                
            analysis_length = time.time() - start_time
            print(f"[INFO] finished video analysis in {analysis_length}")

    
    return results

In [28]:
print(analyze('input/short_hamilton_clip.mp4', 'dossier', 'hamilton', outputs_video=False))

16:17:30 ::   WriteGear   :: CRITICAL :: Compression Mode is disabled, Activating OpenCV built-in Writer!


[5] identified 2 faces at 0.7156069278717041


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'image2 / image2 sequence'


[10] identified 2 faces at 1.3791148662567139
[15] identified 2 faces at 2.287763833999634
[20] identified 1 faces at 2.8923590183258057


16:17:33 ::   WriteGear   :: CRITICAL :: Compression Mode is disabled, Activating OpenCV built-in Writer!


...

[30] identified 2 faces at 4.322631120681763
[35] identified 2 faces at 4.859760999679565
[40] identified 1 faces at 5.284618854522705
[45] identified 3 faces at 5.978823184967041
[50] identified 2 faces at 6.653105020523071
[55] identified 4 faces at 7.609364986419678
[60] identified 6 faces at 9.079757928848267
[65] identified 6 faces at 10.388646125793457
[70] identified 5 faces at 11.360676050186157
[75] identified 2 faces at 11.878797054290771
[80] identified 4 faces at 12.674093961715698


16:17:42 ::   WriteGear   :: CRITICAL :: Compression Mode is disabled, Activating OpenCV built-in Writer!


[85] identified 1 faces at 13.034674882888794


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'image2 / image2 sequence'


[90] identified 1 faces at 13.434889793395996
[95] identified 1 faces at 13.803447008132935
[100] identified 1 faces at 14.191295862197876
[105] identified 1 faces at 14.555557012557983
[110] identified 1 faces at 15.03726601600647
[115] identified 1 faces at 15.402453899383545
[120] identified 1 faces at 15.958757162094116
[125] identified 1 faces at 16.421425819396973
[130] identified 1 faces at 16.772372007369995
[135] identified 1 faces at 17.15442991256714
[140] identified 1 faces at 17.52244806289673
[145] identified 1 faces at 17.89653706550598
[150] identified 1 faces at 18.259632110595703
[155] identified 1 faces at 18.62764310836792
[160] identified 1 faces at 18.999907970428467
[165] identified 1 faces at 19.388819932937622
[170] identified 1 faces at 19.766921997070312
[175] identified 1 faces at 20.151077032089233
[180] identified 1 faces at 20.55998992919922
[185] identified 1 faces at 21.018802881240845
[190] identified 1 faces at 21.498097896575928
[195] identified 1 fa


## Sử dụng với module FaceAnalyzer

Hai function `extract_embedding` và `analyze` ở trên tương ứng với hai method trong class Analyzer, chứa trong module FaceAnalyzer.

Bạn có thể import class Analyzer và khởi tạo class này với câu lệnh sau:

In [9]:
from FaceAnalyzer import Analyzer

analyzer = Analyzer()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

Để thực hiện những thao tác tương tự như hai function ở trên, bạn có thể sử dụng những câu lệnh như sau:

In [10]:
analyzer.embedding_extract("hamilton", "hamilton")

2it [00:00,  3.07it/s]

[DEBUG] extract successful


In [11]:
print(analyzer.analyze('short_hamilton_clip.mp4', 'dossier', 'hamilton', outputs_video=False))

10:58:33 ::   WriteGear   :: CRITICAL :: Compression Mode is disabled, Activating OpenCV built-in Writer!


[5] identified 2 faces at 0.4110867977142334


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'image2 / image2 sequence'


[10] identified 2 faces at 0.8099279403686523
[15] identified 2 faces at 1.1508910655975342
[20] identified 1 faces at 1.4128968715667725


10:58:34 ::   WriteGear   :: CRITICAL :: Compression Mode is disabled, Activating OpenCV built-in Writer!


[25] identified 3 faces at 1.8555388450622559


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'image2 / image2 sequence'


[30] identified 2 faces at 2.2328410148620605
[35] identified 2 faces at 2.6260039806365967
[40] identified 1 faces at 2.9521169662475586
[45] identified 3 faces at 3.4372878074645996
[50] identified 2 faces at 3.777549982070923
[55] identified 4 faces at 4.3036417961120605
[60] identified 6 faces at 5.110954999923706
[INFO] finished video analysis in 6.094574689865112
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/_x/zckfnlk92vq72rvw9szdlzqw0000gn/T/ipykernel_7624/2371828781.py", line 1, in <module>
    print(analyzer.analyze('short_hamilton_clip.mp4', 'dossier', 'hamilton', outputs_video=False))
  File "/Users/thaivu/Documents/vsi-intern/learn-insightface/FaceAnalyzer.py", line 104, in analyze
  File "/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/insightface/app/face_analysis.py", line 75, in get
    model.get(img, face)
  File "/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/insightface/model_zoo/arcface_onnx.py", line 67, in get
    face.embedding = self.get_feat(aimg).flatten()
  File "/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/insightface/model_zoo/arcface_onnx.py", line 84, in get_feat
    ne